# Project Overview

This notebook implements an advanced benchmark of four distinct Vision Language Models (VLMs) on the Hateful Memes Challenge Dataset (HMCD). This version loads the dataset from local files, uses a balanced dataset sample, and evaluates models from four different families to capture classification.

In [ ]:
pip install ollama

In [ ]:
pip install pandas

In [ ]:
pip install scikit-learn


In [160]:
import ollama
import pandas as pd
import os

## Step 1: Load Dataset from Local Files & Prepare Sample

In [161]:
DATASET_FOLDER = 'data' 
ANNOTATION_FILE = os.path.join(DATASET_FOLDER, 'dev.jsonl')
IMG_DIR = os.path.join(DATASET_FOLDER, 'img')

In [176]:
if not os.path.exists(ANNOTATION_FILE):
    print(f"Error'")
else:
    df = pd.read_json(ANNOTATION_FILE, lines=True)

    # Create the full path to each image file
    df['img_path'] = df['img'].apply(lambda x: os.path.join(IMG_DIR, x))

    print("Local dataset loaded successfully.")


Local dataset loaded successfully.


### Balanced Sampling Strategy

In [177]:
df['label_text'] = df['label'].map({0: 'non-hateful', 1: 'hateful'})

In [178]:
hateful_df = df[df['label'] == 1].sample(n=250, random_state=45)

In [179]:
non_hateful_df = df[df['label'] == 0].sample(n=250, random_state=45)

In [180]:
df_sample = pd.concat([hateful_df, non_hateful_df])
df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)
df_sample['img_path'] = df_sample['img_path'].apply(os.path.normpath) # Normalize all image paths to match your OS

In [181]:
df_sample.head()

,id,img,label,text,img_path,label_text
0,85237,img/85237.png,0,these are all the weapons that were seized fro...,data\img\img\85237.png,non-hateful
1,28017,img/28017.png,1,say it! say it ! islam is the religion of peace,data\img\img\28017.png,hateful
2,32875,img/32875.png,0,if he wears number one he's most likely the fa...,data\img\img\32875.png,non-hateful
3,12834,img/12834.png,1,"in west philadelphia, born and raised in the a...",data\img\img\12834.png,hateful
4,43810,img/43810.png,1,how to cook turkey in usa how to cook turkey i...,data\img\img\43810.png,hateful


In [83]:
print(f"Created a balanced test set with {len(df_sample)} samples.")
print("Label distribution:")
print(df_sample['label_text'].value_counts())

Created a balanced test set with 500 samples.
Label distribution:
label_text
non-hateful    250
hateful        250
Name: count, dtype: int64


## Step 2: Playing with the Prompt

In [145]:
def create_zero_shot_prompt():
    """Asks for a direct classification."""
    return (
        "You are an expert content moderation analyst. Classify the provided meme as either 'hateful' or 'non-hateful'.\n"
        "Respond with only a single word: `hateful` or `non-hateful`."
    )

In [126]:
def create_cot_prompt():
    """Guide the model to reason step-by-step before classifying."""
    return (
        "Perform the following steps:\n"
        "1. Briefly describe the visual elements in the image.\n"
        "2. Analyze the text on the meme.\n"
        "3. Consider the combined meaning of the visual elements and the text.\n\n"
        "Explain your step-by-step analysis and classify the meme as 'hateful' or 'non-hateful'. "
        "Conclude your analysis with a single word classification."
    )


In [146]:
prompt_template = create_zero_shot_prompt()
# prompt_template = create_cot_prompt()

In [103]:
def parse_response(response_text):
    cleaned_text = response_text.lower().strip().replace('.', '')
    if 'non-hateful' in cleaned_text:
        return 'non-hateful'
    elif 'hateful' in cleaned_text:
        return 'hateful'
    else:
        return 'error'

In [182]:
#Test Ollama Server

import requests

try:
    r = requests.get("http://localhost:11434")
    print("Ollama server is running")
except Exception as e:
    print("Ollama is not running:", e)

Ollama server is running


In [144]:
# def test_image_only(image_path):
#     import base64, requests

#     with open(image_path, "rb") as img:
#         image_data = base64.b64encode(img.read()).decode("utf-8")

#     response = requests.post(
#         "http://localhost:11434/api/generate",
#         json={
#             "model": "llava:7b",
#             "prompt": "Describe everything in this image.",
#             "images": [image_data],
#             "stream": False
#         }
#     )
#     print(response.json()["response"])

# test_image_only("data\img\98714.png")

 In the image, a person is seen skydiving from a Cessna aircraft. The individual, clad in a black helmet, has their arms outstretched as if embracing the thrill of the jump. The plane, painted in white and blue, is captured mid-flight against a backdrop of a clear blue sky. On the right side of the image, there's a text that humorously suggests that when the dive instructor starts yelling in Arabic, it's time to start crying for your life. The person appears to be having an enjoyable and adrenaline-filled experience, as indicated by their wide grin. 


In [154]:
# Test for single image

# import base64
# import requests

# model_name = "llava:7b"
# image_path = "data\img\\28017.png"  # Use an actual file path here
# prompt = "What is in this image?"

# def test_llava(model_name, image_path, prompt):
#     try:
#         with open(image_path, "rb") as f:
#             image_data = base64.b64encode(f.read()).decode("utf-8")

#         response = requests.post(
#             "http://localhost:11434/api/generate",
#             json={
#                 "model": model_name,
#                 "prompt": prompt,
#                 "images": [image_data],
#                 "stream": False
#             }
#         )

#         print("✅ Response:", response.json()["response"])
#     except Exception as e:
#         print("❌ Error:", e)

# test_llava(model_name, image_path, prompt_template)

✅ Response:  Non-hateful 


In [155]:
import requests
import base64

def classify_with_ollama(model_name, image_path, prompt):
    try:
        # Read and encode the image
        with open(image_path, "rb") as f:
            image_data = base64.b64encode(f.read()).decode("utf-8")

        # Send HTTP request to Ollama API
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model_name,
                "prompt": prompt,
                "images": [image_data],
                "stream": False
            }
        )

        result = response.json()["response"]

        # You can still use parse_response if needed:
        return parse_response(result)

    except Exception as e:
        return f"error: {e}"

In [107]:
# def classify_with_ollama(model_name, image_path, prompt):
#     try:
#         response = ollama.chat(model=model_name, messages=[{'role': 'user', 'content': prompt, 'images': [image_path]}])
#         return parse_response(response['message']['content'])
#     except Exception as e:
#         return 'error'

In [111]:
import base64

In [ ]:
# def image_to_base64(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

In [90]:
from tqdm import tqdm

In [157]:
import os

# Check which files don't exist
missing_files = df_sample[~df_sample['img_path'].apply(os.path.exists)]

if not missing_files.empty:
    print("❌ Missing image files:")
    display(missing_files[['id', 'img_path']])
else:
    print("✅ All image paths are valid.")

❌ Missing image files:


,id,img_path
0,85237,data\img\img/85237.png
1,28017,data\img\img/28017.png
2,32875,data\img\img/32875.png
3,12834,data\img\img/12834.png
4,43810,data\img\img/43810.png
...,...,...
495,95640,data\img\img/95640.png
496,32579,data\img\img/32579.png
497,38095,data\img\img/38095.png
498,30586,data\img\img/30586.png


In [156]:
models_to_test = {
    'llava:7b': classify_with_ollama
}
results_data = {model: [] for model in models_to_test}

for model_name, classification_func in models_to_test.items():
    print(f"\n--- Benchmarking model: {model_name} ---")
    predictions = []
    for index, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc=f"Processing {model_name}"):
        if 'ollama' in classification_func.__name__:
            pred = classification_func(model_name, row['img_path'], prompt_template)
        else:
            pred = classification_func(row['img_path'], prompt_template) ## for gemini or claude
        predictions.append(pred)
    df_sample[f'prediction_{model_name}'] = predictions

print("\n--- Benchmark Complete! ---")
display(df_sample[['id', 'label_text'] + [f'prediction_{model}' for model in models_to_test.keys()]].head())




--- Benchmarking model: llava:7b ---


Processing llava:7b: 100%|██████████| 500/500 [00:00<00:00, 3496.54it/s]


--- Benchmark Complete! ---


,id,label_text,prediction_llava:7b
0,85237,non-hateful,error: [Errno 2] No such file or directory: 'd...
1,28017,hateful,error: [Errno 2] No such file or directory: 'd...
2,32875,non-hateful,error: [Errno 2] No such file or directory: 'd...
3,12834,hateful,error: [Errno 2] No such file or directory: 'd...
4,43810,hateful,error: [Errno 2] No such file or directory: 'd...


In [54]:
ground_truth = df_sample['label_text']


In [106]:
from sklearn.metrics import classification_report

for model_name in models_to_test.keys():
    print(f"\n--- Evaluation Report for: {model_name} ---")
    model_predictions = df_sample[f'prediction_{model_name}']
    report = classification_report(ground_truth, model_predictions, labels=['hateful', 'non-hateful'], zero_division=0)
    print(report)

ModuleNotFoundError: No module named 'numpy.strings'

In [61]:
columns_to_show = ['label_text'] + [col for col in df_sample.columns if 'prediction' in col]
full_results_df = df_sample[columns_to_show]
display(full_results_df)


,label_text,prediction_llava:7b
0,non-hateful,error
1,hateful,error
2,non-hateful,error
3,hateful,error
4,hateful,error
...,...,...
495,hateful,error
496,non-hateful,error
497,non-hateful,error
498,non-hateful,error


In [60]:
full_results_df.to_csv('full_benchmark_results.csv', index=False)